In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 
Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'

In [2]:
Today = '2024-10-31'

print_Deals = True
excel_tofolder_on_Z = True

query = f"""
select MAX("reportDate") AS max_date
from RISKACCESS."sapIcDeals"
where "reportDate" <= TO_DATE('{Today}', 'YYYY-MM-DD')
"""
data_export = export_from_RISKCUSTOM(query)
Today = str(data_export.iloc[0,0])[:10]

Today

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


'2024-10-31'

In [3]:
query = f"""select * from RISKACCESS."sapDeals" where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') """
data_ex_deals_export = export_from_RISKCUSTOM(query)

query = f"""select * from RISKACCESS."sapIcDeals" where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') """
data_in_deals_export = export_from_RISKCUSTOM(query)

data_ex_deals_export.shape, data_in_deals_export.shape

((8640, 204), (1496, 163))

In [4]:
work_data_list = [data_ex_deals_export, data_in_deals_export]

for work_data_index in range(len(work_data_list)):
    data_work = work_data_list[work_data_index]
    data_work = data_work.query("dealStatusName != 'Cancelled' & invoiced == 'Y' & deliveryPeriodTo > @Today").reset_index(drop=True)
    data_work[['sellerName', 'buyerName']] = data_work[['sellerName', 'buyerName']].fillna('')
    data_work['Counterparty'] = data_work['sellerName'] + data_work['buyerName']
    data_work.loc[data_work['actualQuantity'].isna(), 'actualQuantity'] = data_work.loc[data_work['actualQuantity'].isna(), 'BLTonnage']
    if work_data_index == 0:
        price_col = 'conditionPrice'
    else:
        price_col = 'price'
    data_work['Sum_money'] = data_work['actualQuantity'] * data_work[price_col]
    data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'actualQuantity'] = data_work.actualQuantity.abs()
    data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'Sum_money'] = -1 * data_work.Sum_money.abs()
    data_work.loc[data_work.dealTypeName == 'Sales Deal', 'actualQuantity'] = -1 * data_work.actualQuantity.abs()
    data_work.loc[data_work.dealTypeName == 'Sales Deal', 'Sum_money'] = data_work.Sum_money.abs()
    data_work['CompCode'] = merge_Mapping(data_work, col='companyName')
    data_work['Business_segment'] = merge_SalesUnits(data_work, col='CompCode', merge_col='ocpSegment')
    data_work['Cpty_Code'] = merge_Mapping(data_work, col='Counterparty')
    data_work['Cpty_segment'] = merge_SalesUnits(data_work, col='Cpty_Code', merge_col='ocpSegment').fillna('External')
    data_work = Period(data_work, day_for_count=Today, col_with_date='deliveryPeriodTo')
    data_work['holding'] = merge_SalesUnits(data_work, 'CompCode', merge_col='holding')

    data_for_print = data_work[['productTypeName', 'companyName', 'Business_segment', 'Counterparty',  'Cpty_segment', 'currency', 'actualQuantity', 'Sum_money', 'deliveryPeriodTo', 'Days', 'Period', 'holding']]
    if work_data_index == 0:
        work_data_ex_deals = data_for_print
    else:
        work_data_in_deals = data_for_print

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG'
 'EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG' 'EX SUEK AG'
 'EX SUEK AG' 'EX SUEK AG']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projec

In [5]:
if print_Deals == True:
    for group in ['EUROCHEM', 'SUEK']:
        Output_file = "_".join([str(date.today()), group, 'Deals.xlsx'])
        work_data_ex_deals_group = work_data_ex_deals[work_data_ex_deals.holding == group]
        work_data_in_deals_group = work_data_in_deals[work_data_in_deals.holding == group]
        sheet_name_ex = 'EX_Deals'
        sheet_name_in = 'IN_Deals'
        if excel_tofolder_on_Z == True:
            Output_file = Output_path + Output_file
        if len(work_data_ex_deals_group)!=0:
            work_data_ex_deals_group.to_excel(Output_file, sheet_name_ex, index=False)
        elif len(work_data_in_deals_group)!=0:
            new_list(work_data_in_deals_group, Output_file, sheet_name_in, index=False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_23064\2662934976.py:11: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  work_data_ex_deals_group.to_excel(Output_file, sheet_name_ex, index=False)
